In [126]:
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
from utils import isWorkday, estEndDate
import pyodbc
import random

In [127]:
server = 'LAPTOP-2NSE0JH1\SQLEXPRESS'
database = 'dummy'

conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
conn_str

'DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-2NSE0JH1\\SQLEXPRESS;DATABASE=dummy;Trusted_Connection=yes;'

In [238]:
try:
    # Establish connection
    conn = pyodbc.connect(conn_str)

    # Create a cursor
    cursor = conn.cursor()

    # Execute query to extract data from Task table
    task_query = "SELECT * FROM Task"
    tasks = pd.read_sql(task_query, conn)

    # Execute query to extract data from Project table
    project_query = "SELECT * FROM Project"
    projects = pd.read_sql(project_query, conn)

    # Close connection
    conn.close()

except pyodbc.Error as e:
    print("Error connecting to SQL Server:", e)

C:\Users\Acer\AppData\Local\Temp\ipykernel_1932\137494329.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tasks = pd.read_sql(task_query, conn)
C:\Users\Acer\AppData\Local\Temp\ipykernel_1932\137494329.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  projects = pd.read_sql(project_query, conn)


In [239]:
tasks['ParentTaskID'] = tasks['ParentTaskID'].astype('Int64')  # Convert to nullable integer
tasks['ActualStartDate'] = tasks['ActualStartDate'].astype('object')  # Convert to object
tasks['ActualEndDate'] = tasks['ActualEndDate'].astype('object')  # Convert to object

In [240]:
# tasks.to_csv('task_10.csv', index=False)
# projects.to_csv('project_10.csv', index=False)

In [241]:
def isParentCompleted(task):
    if pd.isnull(task['ParentTaskID']):
        return True
    else:
        parent_task = tasks[tasks['ID'] == task['ParentTaskID']].iloc[0]
        return parent_task['Status'] == 'Completed'
    
def delay(task):
    if task['Priority'] == 'Critical':
        return random.randint(1,14) == 1
    else :
        return random.randint(1,10) == 1

In [242]:
curr_date = datetime.strptime('2024-03-12','%Y-%m-%d').date()
task_report = []
project_report = []

while ~tasks['Status'].eq('Completed').all():
    task_today = tasks[(tasks['StartDate'] <= curr_date) & (tasks['Status']!='Completed')]['ID'].tolist()

    for idx in task_today:
        task = tasks.loc[tasks['ID']==idx].iloc[0]
        workday = projects.loc[projects['ID'] == task['ProjectID']].iloc[0]['Workday']
        if isParentCompleted(task) and isWorkday(curr_date, workday):
            if task['Status'] == 'Not Started':
                task['ActualStartDate'] = str(curr_date)
                task['Status'] = 'On Progress'
                
            if delay(task):
                task['Progress'] += 0
            else:
                task['Progress'] += 1
                
            if task['Progress'] >= task['Duration']:
                task['ActualEndDate'] = curr_date
                task['Status'] = 'Completed'
            
            if curr_date >= task['EndDate'] and task['Status'] == 'On Progress':
                task['Status'] = 'Delayed'
                task['Priority'] = 'Critical'
            
            tasks.loc[tasks['ID']==idx] = task.values
            
            task_report.append({
                'Date': curr_date,
                'ID': task['ID'],
                'Name': task['Name'],
                'StartDate': task['StartDate'],
                'EndDate': task['EndDate'],
                'Cost': task['Cost'],
                'Priority': task['Priority'],
                'Progress': task['Progress'],
                'ProjectID' : task['ProjectID'],
                'Status': task['Status'],
                'Duration': task['Duration'],
                'ActualStartDate': task['ActualStartDate'],
                'ActualEndDate': task['ActualEndDate']
            })
            
    for pid in projects['ID'].tolist():
        project_task = tasks[tasks['ProjectID']==pid]
        if ~project_task['Status'].eq('Not Started').all() and ~project_task['Status'].eq('Completed').all():
            project_report.append({
                'Date' : curr_date,
                'ProjectID' : pid,
                'TotalTask' : len(project_task),
                'StartedTask' : len(project_task[project_task['Status']!='Not Started']),
                'OnGoingTask' : len(project_task[(project_task['Status']!='Not Started') & (project_task['Status']!='Completed')]),
                'DelayedTask' : len(project_task[(project_task['Status']=='Delayed')]),
                'Completedtask' : len(project_task[(project_task['Status']=='Completed')]),
                'WorkDay' : project_task[(project_task['Status']!='Not Started')]['Progress'].sum(),
                'TotalSpent' : project_task[(project_task['Status']=='Completed')]['Cost'].sum()
            })

    curr_date += timedelta(days=1)

task_reports = pd.DataFrame(task_report)
project_reports = pd.DataFrame(project_report)
task_reports['ActualEndDate'] = task_reports.groupby('ID')['ActualEndDate'].bfill()
project_dates = tasks.groupby('ProjectID').agg({'StartDate': 'min', 'EndDate': 'max', 'ActualStartDate':'min','ActualEndDate':'max'}).reset_index()
project_reports = pd.merge(project_reports, project_dates, on='ProjectID', how='left')

print(f'Complete all task at {curr_date}')

Complete all task at 2024-07-08


In [247]:
task_reports.to_csv('task_report_10.csv',index=False)
project_reports.to_csv('project_report_10.csv',index=False)